In [ ]:
import numpy as np 
import pandas as pd 

df_train = pd.read_csv('goodreads_train.csv')
df_test = pd.read_csv('goodreads_test.csv')
df_sub = pd.read_csv('goodreads_sample_submission.csv')

# Delete dublicate reviews and review_id
df_train.drop_duplicates(subset=["review_id"], inplace=True, keep='first')
df_train.drop_duplicates(subset=["review_text"], inplace=True, keep='first')

# Delete reviews with less than 5 letters.
indexReviewText = df_train.loc[ df_train['review_text'].str.len() <= 5].index
df_train.drop(indexReviewText , inplace=True)

# # Delete needless columns
df_train.drop(columns=['book_id','review_text','n_votes','n_comments','user_id','date_added','date_updated','read_at','started_at'])

# Clean the review data which consists of word greater than 30
get_word_length = lambda phrase: np.mean(list(map(len, phrase.split())))
condition = df_train.review_text.apply(get_word_length) > 30
index = df_train.index
bad_word_indices = index[condition].tolist()
df_train.drop(bad_word_indices,inplace=True)

In [ ]:
df_train['rating'].value_counts()
df_train = df_train.groupby('rating').head(28555)
condition = df_train.review_text.apply(len)

In [ ]:
def simple_linear_regression(input_feature, output):
    count = len(input_feature)
    numerator = ((input_feature*output).sum()) - ((1/count)*(input_feature.sum()*output.sum()))
    denominator = (input_feature*input_feature).sum() - (1/count)*(input_feature.sum()*input_feature.sum())
    slope = numerator/denominator
    intercept = output.mean() - (slope *(input_feature.mean()))
    return (intercept, slope)

intercept, slope = simple_linear_regression(condition, df_train['rating'])

def get_regression_predictions(input_feature, intercept, slope):
    predicted_values = intercept + slope*input_feature
    return predicted_values

In [ ]:
estimated_rating = []

for reviews in df_test['review_text']:
    rating = get_regression_predictions(len(reviews), intercept, slope)
    estimated_rating.append(round(rating))
    
df_sub.rating = estimated_rating
df_sub.to_csv("submission.csv", index=False)